# Cycling in numbers - A Case Study of Cycle Paths in Rhine-Kreis Neuss

## Description

Five Counting stations have been permanently documenting cycling traffic on central roads since 2016 in Cycle paths in the Rhine-Kreis Neuss. The daily measurement of cycling traffic is done with the help of induction loops laid in the ground. With the permanent collection of data can gain insights on the daily, weekly and annual cycles and on it building long-term cycling developments over several years.

More details on the data source here: https://data.europa.eu/data/datasets/eco-counter-data-rhein-kreis-neuss?locale=en

# Streaming Simulation - Batching for real time monitoring

Once the data is ingested in real-time, Spark Structured Streaming processes and aggregates the information for deeper analysis. A Kafka consumer reads data from the streaming topic, transforming raw JSON messages into structured DataFrames. Hourly aggregations provide insights into bicycle traffic fluctuations throughout the day, while station-wise grouping enables location-based analysis.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window
from pyspark.sql.types import StructType, IntegerType, TimestampType, StringType, DoubleType
import findspark

findspark.init()
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2" pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2 pyspark-shell'

# Define schema based on the structure of your data
schema = StructType() \
    .add("Id", IntegerType(), nullable=True) \
    .add("Date", TimestampType(), nullable=True) \
    .add("Count", IntegerType(), nullable=True) \
    .add("Status", StringType(), nullable=True) \
    .add("Channel_Id", IntegerType(), nullable=True) \
    .add("Counting_Station", StringType(), nullable=True) \
    .add("Coordinates", StringType(), nullable=True) \
    .add("Year", IntegerType(), nullable=True) \
    .add("Month", IntegerType(), nullable=True) \
    .add("Day", IntegerType(), nullable=True) \
    .add("Hour", IntegerType(), nullable=True) \
    .add("Day_of_Week", IntegerType(), nullable=True) \
    .add("Latitude", DoubleType(), nullable=True) \
    .add("Longitude", DoubleType(), nullable=True)

# Initialize Spark session
spark = SparkSession.builder \
    .appName("EcoCounterStreaming") \
    .getOrCreate()

# Read data from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "cycling_data") \
    .load()
    #.option("startingOffsets", "latest") \
    

# Parse JSON data from Kafka message
parsed_df = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Convert 'Date' string to Timestamp
parsed_df = parsed_df.withColumn("Date", col("Date").cast(TimestampType()))

# Aggregate data per hour based on 'Date'
agg_df = parsed_df \
    .groupBy(window("Date", "1 hour")) \
    .sum("Count")

# Output to console for testing
query = agg_df.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination()

25/03/17 18:21:32 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
25/03/17 18:21:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fed46127-3147-48cf-b649-89b24a4ab7e1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.2 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 2439ms :: artifacts dl 80ms
	:

-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+
|window|sum(Count)|
+------+----------+
+------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+----------+
|              window|sum(Count)|
+--------------------+----------+
|{2016-01-27 01:00...|         0|
|{2016-01-26 22:00...|         0|
|{2016-01-26 23:00...|         0|
|{2016-01-27 02:00...|         0|
|{2016-01-27 04:00...|         0|
|{2016-01-27 03:00...|         0|
|{2016-01-27 00:00...|         0|
+--------------------+----------+



...

-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------+----------+
|              window|sum(Count)|
+--------------------+----------+
|{2016-01-27 15:00...|        12|
|{2016-01-27 16:00...|         9|
|{2016-01-27 18:00...|         7|
|{2016-01-27 17:00...|        23|
+--------------------+----------+



-------------------------------------------
Batch: 6
-------------------------------------------
+--------------------+----------+
|              window|sum(Count)|
+--------------------+----------+
|{2016-01-27 21:00...|         0|
|{2016-01-27 20:00...|         2|
|{2016-01-27 19:00...|         8|
|{2016-01-27 18:00...|         7|
+--------------------+----------+



-------------------------------------------
Batch: 7
-------------------------------------------
+--------------------+----------+
|              window|sum(Count)|
+--------------------+----------+
|{2016-01-27 21:00...|         0|
|{2016-01-28 00:00...|         0|
|{2016-01-27 23:00...|         0|
|{2016-01-27 22:00...|         0|
+--------------------+----------+



ERROR:root:Exception while sending command.=====>               (141 + 2) / 200]
Traceback (most recent call last):
  File "/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=59>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  

Py4JError: An error occurred while calling o67.awaitTermination